In [1]:
import os
import re
import pdb
import string
import numpy as np
import pandas as pd
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [62]:
path_data = "news_summary_small.csv"

In [63]:
df_text_file = pd.read_csv(path_data)

In [4]:
df_text_file.head()

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [5]:
df_text_file.headlines = df_text_file.headlines.str.lower()
df_text_file.text = df_text_file.text.str.lower()

In [6]:
df_text_file.head()

,headlines,text
0,upgrad learner switches to career in ml & al w...,"saurav kant, an alumnus of upgrad and iiit-b's..."
1,delhi techie wins free food from swiggy for on...,kunal shah's credit card bill payment platform...
2,new zealand end rohit sharma-led india's 12-ma...,new zealand defeated india by 8 wickets in the...
3,aegon life iterm insurance plan helps customer...,"with aegon life iterm insurance plan, customer..."
4,"have known hirani for yrs, what if metoo claim...",speaking about the sexual harassment allegatio...


In [74]:
df_text_file.loc[800, ["headlines", "text"]].values

array(['Mercedes-Benz India launches the new V-Class',
       "Celebrating its 25th year, Mercedes-Benz India is set to redefine India's luxury space in the automotive segment by launching the new V-Class. The V-Class is powered by a 2.1-litre BS VI diesel engine that generates 120kW power, 380Nm torque and can go from 0-100km/h in 10.9 seconds. It features LED headlamps, multi-functional steering wheel and 17-inch alloy wheels."],
      dtype=object)

In [9]:
lengths_text = df_text_file.text.apply(len)

In [10]:
max(lengths_text)

450

In [11]:
max(df_text_file.headlines.apply(len))

72

In [12]:
dataset = list(zip(df_text_file.text.values, df_text_file.headlines.values))

In [13]:
# make vocab dictionary

In [16]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

In [17]:
for line in dataset:
    input_text, target_text = list(line[0]), list(line[1])
    target_text = ['BEGIN_'] + target_text + ['_END']
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    
    for character in input_text:
        if character not in input_chars:
            input_chars.add(character)
    for character in target_text:
        if character not in target_chars:
            target_chars.add(character)

In [18]:
input_chars

{'\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 '@',
 '[',
 ']',
 '^',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\x80',
 '\x81',
 '\x82',
 '\x83',
 '\x84',
 '\x87',
 '\x89',
 '\x8b',
 '\x8f',
 '\x90',
 '\x93',
 '\x94',
 '\x96',
 '\x97',
 '\x98',
 '\x99',
 '\x9c',
 '\x9d',
 '\x9e',
 '\x9f',
 '\xa0',
 '¡',
 '£',
 '¤',
 '¦',
 '¨',
 '©',
 '«',
 '¬',
 '\xad',
 '®',
 '°',
 '±',
 '³',
 '¶',
 '¹',
 'º',
 '¼',
 'â',
 'ã',
 'ä',
 'å'}

In [19]:
target_chars

{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'BEGIN_',
 '_END',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\x80',
 '\x82',
 '\x87',
 '\x89',
 '\x8b',
 '\x96',
 '\x98',
 '\x99',
 '\x9c',
 '\x9d',
 '\xa0',
 '£',
 '¤',
 '¦',
 '§',
 '©',
 '¬',
 '°',
 '¹',
 'º',
 'â',
 'ã',
 'ä'}

In [20]:
# Add <unk> and <pad>

In [23]:
input_chars.add("<unk>")
input_chars.add("<pad>")
target_chars.add("<pad>")

In [24]:
input_chars = sorted(input_chars)
target_chars = sorted(target_chars)

In [25]:
len(input_chars)

105

In [26]:
len(target_chars)

81

In [27]:
human_vocab = dict(zip(input_chars, range(len(input_chars))))

In [28]:
machine_vocab = dict(zip(target_chars, range(len(target_chars))))

In [53]:
inv_machine_vocab = dict(enumerate(sorted(machine_vocab)))

In [30]:
def string_to_int(string_in, length, vocab):
    """
    Converts all strings in the vocabulary into a list of integers representing the positions of the
    input string's characters in the "vocab"

    Arguments:
    string -- input string
    length -- the number of time steps you'd like, determines if the output will be padded or cut
    vocab -- vocabulary, dictionary used to index every character of your "string"

    Returns:
    rep -- list of integers (or '<unk>') (size = length) representing the position of the string's character in the vocabulary
    """

    #make lower to standardize
    string_in = string_in.lower()
    string_in = string_in.replace(',','')

    if len(string_in) > length:
        string_in = string_in[:length]

    rep = list(map(lambda x: vocab.get(x, '<unk>'), string_in))

    if len(string_in) < length:
        rep += [vocab['<pad>']] * (length - len(string_in))
        
    return rep

In [31]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):

    X, Y = zip(*dataset)
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    print("X shape from preprocess: {}".format(X.shape))

    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

In [32]:
Tx = 460
Ty = 75
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

X shape from preprocess: (10000, 460)


In [33]:
X.shape

(10000, 460)

In [35]:
Y.shape

(10000, 75)

In [37]:
Xoh.shape

(10000, 460, 105)

In [38]:
Yoh.shape

(10000, 75, 81)

In [39]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index].shape)
print("Target after preprocessing (indices):", Y[index].shape)
print()
print("Source after preprocessing (one-hot):", Xoh[index].shape)
print("Target after preprocessing (one-hot):", Yoh[index].shape)

Source date: saurav kant, an alumnus of upgrad and iiit-b's pg program in machine learning and artificial intelligence, was a sr systems engineer at infosys with almost 5 years of work experience. the program and upgrad's 360-degree career support helped him transition to a data scientist at tech mahindra with 90% salary hike. upgrad's online power learning has powered 3 lakh+ careers.
Target date: upgrad learner switches to career in ml & al with 90% salary hike

Source after preprocessing (indices): (460,)
Target after preprocessing (indices): (75,)

Source after preprocessing (one-hot): (460, 105)
Target after preprocessing (one-hot): (75, 81)


In [40]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm

In [42]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [43]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [44]:
def one_step_attention(h, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "h" of the Bi-LSTM.
    
    Arguments:
    h -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_h)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([h, s_prev])
    
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e.
    e = densor1(concat)
    
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies.
    energies = densor2(e)
    
    # Use "activator" on "energies" to compute the attention weights "alphas"
    alphas = activator(energies)
    
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas, h])
    
    return context

In [45]:
n_h = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [46]:
def model(Tx, Ty, n_h, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_h -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size), name="input_first")
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    
    # Define your pre-attention Bi-LSTM. Remember to use return_sequences=True.
    a = Bidirectional(LSTM(n_h, return_sequences=True))(X)
    
    # Iterate for Ty steps
    for t in range(Ty):
    
        # Perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(h, s)
        
        # Apply the post-attention LSTM cell to the "context" vector.
        # Pass: initial_state = [hidden state, cell state]
        s, _, c = post_activation_LSTM_cell(context, initial_state = [s,c])
        
        #Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        #Append "out" to the "outputs" list
        outputs.append(out)
    
    # Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

In [47]:
model = model(Tx, Ty, n_h, n_s, len(human_vocab), len(machine_vocab))

In [48]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_first (InputLayer)        (None, 460, 105)     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 460, 64)      35328       input_first[0][0]                
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 460, 64)      0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [49]:

opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [50]:
s0 = np.zeros((10000, n_s))
c0 = np.zeros((10000, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [51]:
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)

Epoch 1/1
10000/10000 [==============================] - 561s 56ms/step - loss: 227.1987 - dense_5_loss: 1.6683 - dense_5_acc: 0.0401 - dense_5_acc_1: 0.0720 - dense_5_acc_2: 0.0312 - dense_5_acc_3: 0.0371 - dense_5_acc_4: 0.0357 - dense_5_acc_5: 0.0364 - dense_5_acc_6: 0.0423 - dense_5_acc_7: 0.0373 - dense_5_acc_8: 0.0297 - dense_5_acc_9: 0.0283 - dense_5_acc_10: 0.0255 - dense_5_acc_11: 0.0178 - dense_5_acc_12: 0.0160 - dense_5_acc_13: 0.0188 - dense_5_acc_14: 0.0174 - dense_5_acc_15: 0.0154 - dense_5_acc_16: 0.0129 - dense_5_acc_17: 0.0117 - dense_5_acc_18: 0.0087 - dense_5_acc_19: 0.0089 - dense_5_acc_20: 0.0077 - dense_5_acc_21: 0.0065 - dense_5_acc_22: 0.0070 - dense_5_acc_23: 0.0078 - dense_5_acc_24: 0.0057 - dense_5_acc_25: 0.0042 - dense_5_acc_26: 0.0068 - dense_5_acc_27: 0.0030 - dense_5_acc_28: 0.0039 - dense_5_acc_29: 0.0041 - dense_5_acc_30: 0.0019 - dense_5_acc_31: 0.0017 - dense_5_acc_32: 0.0027 - dense_5_acc_33: 0.0022 - dense_5_acc_34: 0.0025 - dense_5_acc_35: 0.0031 

In [68]:
EXAMPLES = ["Last night a meteorite was seen flying near the earth's moon."]
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source[np.newaxis, :]
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: Last night a meteorite was seen flying near the earth's moon.
output: aaa           <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
